## Rotor

In [ ]:
class Rotor:
    def __init__(self, wiring, notch, position=0):
        self.wiring = wiring
        self.notch = notch
        self.position = position % 26
        self.alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    def encode_forward(self, c):
        idx = (self.alphabet.index(c) + self.position) % 26
        letter = self.wiring[idx]
        output = self.alphabet[(self.alphabet.index(letter) - self.position) % 26]
        debugger.debug(f"Rotor({self.notch})", f"{c} -> {letter} -> {output}")
        return output

    def encode_backward(self, c):
        idx = (self.alphabet.index(c) + self.position ) % 26
        letter = self.alphabet[self.wiring.index(self.alphabet[idx])]
        output = self.alphabet[(self.alphabet.index(letter) - self.position) % 26]
        debugger.debug(f"Rotor({self.notch})", f"{c} -> {letter} -> {output}")
        return output

    def step(self):
        self.position = (self.position + 1) % 26
        debugger.debug(f"Rotor({self.notch})", f"Girou!")
        return self.alphabet[self.position] == self.notch

## Refletor

In [ ]:
class Reflector:
    def __init__(self, wiring):
        self.wiring = wiring
        self.alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    def reflect(self, c):
        idx = self.alphabet.index(c)
        output = self.wiring[idx]
        debugger.debug(f"Refletor", f"{c} -> {output}")
        return output

## Plugboard

In [ ]:
class Plugboard:
    def __init__(self, wiring=''):
        self.alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        self.wiring = {}
        for c in self.alphabet:
            self.wiring[c] = c
        pairs = wiring.upper().split()
        for pair in pairs:
            if len(pair) == 2:
                a, b = pair[0], pair[1]
                self.wiring[a] = b
                self.wiring[b] = a

    def swap(self, c):
        output = self.wiring[c]
        debugger.debug("Plugboard", f"{c} -> {output}")
        return output



## Máquina Enigma

In [ ]:
class EnigmaMachine:
    def __init__(self, rotors, reflector, ring_settings=[0,0,0], rotor_positions=[0,0,0], plugboard_wiring=''):
        self.alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        self.rotors = []
        for i, rotor_name in enumerate(rotors):
            wiring = rotor_wirings[rotor_name]
            notch = rotor_notches[rotor_name]
            position = rotor_positions[i]
            rotor = Rotor(wiring, notch, position)
            self.rotors.append(rotor)

        self.reflector = Reflector(reflector)

        self.plugboard = Plugboard(plugboard_wiring)

    def step_rotors(self):

        step_next = self.rotors[2].step()

        if step_next or self.alphabet[self.rotors[1].position] == self.rotors[1].notch:
            step_next = self.rotors[1].step()
        else:
            step_next = False

        if step_next:
            self.rotors[0].step()

    def encrypt_letter(self, c):
        if c not in self.alphabet:
            return c

        self.step_rotors()

        c = self.plugboard.swap(c)

        for rotor in reversed(self.rotors):
            c = rotor.encode_forward(c)

        c = self.reflector.reflect(c)

        for rotor in self.rotors:
            c = rotor.encode_backward(c)

        c = self.plugboard.swap(c)

        return c

    def encrypt_message(self, message):
        result = ''
        for c in message.upper():
            if c in self.alphabet:
                result += self.encrypt_letter(c)
            else:
                result += c
        return result

## Aplicação da Enigma

In [ ]:
class Debugger:
  def __init__(self):
      self.debugs = []
      self.rotor_notches = {
        'Q': 'I',
        'E': 'II',
        'V': 'III',
        'J': 'IV',
        'Z': 'V'
      }

  def debug(self, component, action):
    for char in component:
      if char in self.rotor_notches:
        component = component.replace(char, self.rotor_notches[char])
    debug = f"{component}: {action}"
    self.debugs.append(debug)
    return debug

  def show(self):
    return self.debugs

debugger = Debugger()

In [ ]:
rotor_wirings = {
    'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ',
    'II':'AJDKSIRUXBLHWTMCQGZNPYFVOE',
    'III':   'BDFHJLCPRTXVZNYEIWGAKMUSQO',
    'IV':    'ESOVPZJAYQUIRHXLNFTGKDCMWB',
    'V':     'VZBRGITYUPSDNHLXAWMJQOFECK'
}

rotor_notches = {
    'I': 'Q',
    'II': 'E',
    'III': 'V',
    'IV': 'J',
    'V': 'Z'
}

reflector_wirings = {
    'B': 'YRUHQSLDPXNGOKMIEBFZCWVJAT',
    'C': 'FVPJIAOYEDRZXWGCTKUQSBNMHL'
}

if __name__ == '__main__':
    print('Enigma Machine\n Configuração:')
    machine = EnigmaMachine(
        rotors=[input('rotor 1: '), input('rotor 2: '), input('rotor 3: ')],
        reflector= reflector_wirings[input('reflector: ')],
        rotor_positions=[int(input('posição 1: ')), int(input('posição 2: ')), int(input('posição 3: '))],
        plugboard_wiring=input('plugging:')
    )
    texto_claro = ''
    while texto_claro!='!':
        texto_claro = input('message:')
        texto_cifrado = machine.encrypt_message(texto_claro)
        print('Texto cifrado:', texto_cifrado)

    debugs = debugger.show()
    for debug in debugs:
      if "Girou" in debug: print()
      print(debug)

Enigma Machine
 Configuração:
rotor 1: I
rotor 2: II
rotor 3: III
reflector: C
posição 1: 0
posição 2: 0
posição 3: 0
plugging:Ar gk ox
message:Testando
Texto cifrado: DGCPKJGS
message:!
Texto cifrado: !

Rotor(III): Girou!
Plugboard: T -> T
Rotor(III): T -> K -> J
Rotor(II): J -> B -> B
Rotor(I): B -> K -> K
Refletor: K -> R
Rotor(I): R -> X -> X
Rotor(II): X -> I -> I
Rotor(III): I -> E -> D
Plugboard: D -> D

Rotor(III): Girou!
Plugboard: E -> E
Rotor(III): E -> C -> A
Rotor(II): A -> A -> A
Rotor(I): A -> E -> E
Refletor: E -> I
Rotor(I): I -> V -> V
Rotor(II): V -> X -> X
Rotor(III): X -> M -> K
Plugboard: K -> G

Rotor(III): Girou!
Plugboard: S -> S
Rotor(III): S -> M -> J
Rotor(II): J -> B -> B
Rotor(I): B -> K -> K
Refletor: K -> R
Rotor(I): R -> X -> X
Rotor(II): X -> I -> I
Rotor(III): I -> F -> C
Plugboard: C -> C

Rotor(III): Girou!
Plugboard: T -> T
Rotor(III): T -> S -> O
Rotor(II): O -> M -> M
Rotor(I): M -> O -> O
Refletor: O -> G
Rotor(I): G -> F -> F
Rotor(II): F -> W